# Clustering Consulting Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

In [1]:
import findspark, pyspark
findspark.find()

'C:\\Bigdata\\spark-2.4.5-bin-hadoop2.7'

In [2]:
from pyspark import SparkContext, SparkConf 
from pyspark.sql import SparkSession 
spark = SparkSession.builder.master('local').appName('cluster').getOrCreate()
dataset = spark.read.csv("hack_data.csv", inferSchema=True, header=True)
dataset.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [10]:
dataset.head()

Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)

In [4]:
dataset.describe().show()

+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|       null|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022|       null| 13.41106336843464|
|    min|                    1.0|              10.0|                 0|              1.0|               6.0|Afghanistan|              40.0|
|    max|           

In [13]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
dataset.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [32]:
assembler = VectorAssembler(inputCols=[  
     'Session_Connection_Time',
     'Bytes Transferred',
     'Kali_Trace_Used',
     'Servers_Corrupted',
     'Pages_Corrupted',
     'WPM_Typing_Speed'  ], outputCol='features')
final_data = assembler.transform(dataset)
final_data.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed',
 'features']

In [33]:
from pyspark.ml.feature import StandardScaler

# withStd표준편차랑 withMean평균값을 가지고
scaler      = StandardScaler( inputCol='features', 
                              outputCol='scaledFeatures',
                              withStd=True, withMean=True )
scalerModel = scaler.fit(final_data)
final_data  = scalerModel.transform(final_data)
final_data.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed',
 'features',
 'scaledFeatures']

In [89]:
from pyspark.ml.clustering import KMeans

kmeans = KMeans(featuresCol='scaledFeatures', k=3)
model  = kmeans.fit(final_data)
wssse  = model.computeCost(final_data)
print('Within Set Sum of Squared Errors : ' + str(wssse))

Within Set Sum of Squared Errors : 434.149289871582


## 군집중심
- 아래와 같이 3개의 군집중심이 있는걸로 보아 군집은 3개다

In [59]:
centers = model.clusterCenters()
print('cluster Centers:')
for center in centers:
    print(center)

cluster Centers:
[ 0.86984075  0.80244614  0.02990384  0.91949279  0.97535442 -0.99101306]
[-0.912278   -0.74172619  0.97486534 -0.91242187 -0.98548076  1.01576916]
[-0.82790871 -0.86244322 -1.02271149 -0.92647953 -0.96534864  0.96655167]


In [86]:
result = model.transform(final_data)
result.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- scaledFeatures: vector (nullable = true)
 |-- prediction: integer (nullable = false)



In [88]:
result.select('prediction').show(5)

+----------+
|prediction|
+----------+
|         1|
|         2|
|         1|
|         1|
|         2|
+----------+
only showing top 5 rows



In [92]:
# 군집 k = 3개으로 했는데 show(200)까지는 1, 2 밖에
# 안나와서 0인값을 필터하여 확인해보았다
result.filter(result['prediction']==0).show(5)

+-----------------------+-----------------+---------------+-----------------+---------------+------------------+----------------+--------------------+--------------------+----------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|          Location|WPM_Typing_Speed|            features|      scaledFeatures|prediction|
+-----------------------+-----------------+---------------+-----------------+---------------+------------------+----------------+--------------------+--------------------+----------+
|                   46.0|           479.77|              1|             7.58|           14.0|     Liechtenstein|           45.91|[46.0,479.77,1.0,...|[1.13506461197456...|         0|
|                   35.0|           871.76|              1|             7.38|           15.0|        Mauritania|           45.08|[35.0,871.76,1.0,...|[0.35426937056012...|         0|
|                   42.0|           797.74|              0|             7.58|        

In [93]:
result.filter(result['prediction']==0).select('prediction').show(5)

+----------+
|prediction|
+----------+
|         0|
|         0|
|         0|
|         0|
|         0|
+----------+
only showing top 5 rows



In [62]:
# 문제에서 3가지 비슷한 비율의 해커가 있다고 해서 비슷한지 확인해보았다
result.groupby('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   83|
|         2|   84|
|         0|  167|
+----------+-----+



In [94]:
# 아니므로 k=3은 아니다. k=2를 시도해보자

In [187]:
from pyspark.ml.clustering import KMeans

kmeans = KMeans(featuresCol='scaledFeatures', k=3)
kmeans2 = KMeans(featuresCol='scaledFeatures', k=2)

model  = kmeans.fit(final_data)
result = model.transform(final_data)

model2  = kmeans2.fit(final_data)
result2 = model2.transform(final_data)

wssse  = model.computeCost(final_data)
wssse2  = model2.computeCost(final_data)

print('Within Set Sum of Squared Errors\n@k=3 : {}\n@k=2 : {} '.format(str(wssse),str(wssse2)))

Within Set Sum of Squared Errors
@k=3 : 434.149289871582
@k=2 : 601.7707512676687 


In [188]:
centers = model.clusterCenters()
print('cluster Centers:')
for center in centers:
    print(center)

cluster Centers:
[ 0.86984075  0.80244614  0.02990384  0.91949279  0.97535442 -0.99101306]
[-0.912278   -0.74172619  0.97486534 -0.91242187 -0.98548076  1.01576916]
[-0.82790871 -0.86244322 -1.02271149 -0.92647953 -0.96534864  0.96655167]


In [189]:
result2.groupby('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



## k값 컬럼 붙이기join(how='outer')
1. k=3 또는 k=2 컬럼명, prediction 내용으로된 컬럼 추가
1. feature 와 scaledFeatures, prediction를 제거
- drop_column_list = ["drop_column"]
- df = df.select([column for column in df.columns if column not in drop_column_list]) 

In [190]:
result = result.withColumn('k=3', result.prediction)
result.show(2)

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+--------------------+----------+---+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|            features|      scaledFeatures|prediction|k=3|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+--------------------+----------+---+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|           72.37|[8.0,391.09,1.0,2...|[-1.5622280401844...|         1|  1|
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|[20.0,720.99,0.0,...|[-0.7104514131868...|         2|  2|
|                   31.0|           356.32|            

In [192]:
drop_column_list = ["features", "scaledFeatures", "prediction"]
df1 = result.select([column for column in result.columns if column not in drop_column_list])
df1.show(2)

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+---+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|k=3|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+---+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|           72.37|  1|
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|  2|
|                   31.0|           356.32|              1|             3.71|            8.0|             Tokelau|           70.58|  1|
|                    2.0|           228.08|              1|             2.48|            8.0|             Bolivia|            70.8|  1|
|                   20.0|            408.5|     

In [193]:
result2 = result2.withColumn('k=2', result2.prediction)
result2.show(2)

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+--------------------+----------+---+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|            features|      scaledFeatures|prediction|k=2|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+--------------------+----------+---+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|           72.37|[8.0,391.09,1.0,2...|[-1.5622280401844...|         1|  1|
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|[20.0,720.99,0.0,...|[-0.7104514131868...|         1|  1|
|                   31.0|           356.32|            

In [194]:
drop_column_list = ["features", "scaledFeatures", "prediction"]
df2 = result2.select([column for column in result2.columns if column not in drop_column_list])
df2.show(2)

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+---+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|k=2|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+---+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|           72.37|  1|
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|  1|
|                   31.0|           356.32|              1|             3.71|            8.0|             Tokelau|           70.58|  1|
|                    2.0|           228.08|              1|             2.48|            8.0|             Bolivia|            70.8|  1|
|                   20.0|            408.5|     

In [198]:
df =df1.join(df2, on=['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed'], how='outer')

In [199]:
df.show(5)

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+---+---+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|k=3|k=2|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+---+---+
|                   35.0|           717.42|              0|             5.08|           14.0|   Brunei Darussalam|           46.92|  0|  0|
|                   46.0|           783.48|              1|             7.82|           14.0|British Virgin Is...|           43.09|  0|  0|
|                    8.0|           365.62|              0|             1.56|            7.0|South Georgia and...|           69.22|  2|  1|
|                   28.0|           336.36|              1|             3.06|            8.0|       Faroe Islands|           69.13|  1|  1|
|                   